difference between memory-schema-profile-1 and memory-schema-profile-2 is in memory-schema-profile-1 using with_structured_output to invoke the model. in memory-schema-profile-2, we will use original invoke and use TrustCall. TrustCall is an open-source library for updating JSON schemas.

In [1]:
import openai
import os

from dotenv import find_dotenv, load_dotenv
from pydantic import BaseModel, Field
from trustcall import create_extractor
from typing import List

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, MessagesState, StateGraph, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

In [2]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
# example using TrustCall

conversation = [
    HumanMessage(content="Hi, i am Yoona."), 
    AIMessage(content="Nice to meet you, Yoona."), 
    HumanMessage(content="I love 2nd generation k-pop group and AI enthusiast.")
]

class UserProfile(BaseModel):
    """User profile schema with typed fields.
    """
    user_name: str = Field(description="The user's preferred name")
    interests: List[str] = Field(description="A list of the user's interests")

trustcall_extractor = create_extractor(
    llm,
    tools=[UserProfile],
    tool_choice="UserProfile"
)

system_msg = "Extract the user profile from the following conversation"

result = trustcall_extractor.invoke({"messages": [SystemMessage(content=system_msg)]+conversation})

for m in result["messages"]: 
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  UserProfile (call_vmlqGOnGnIRTXP2mzOZEPiOI)
 Call ID: call_vmlqGOnGnIRTXP2mzOZEPiOI
  Args:
    user_name: Yoona
    interests: ['2nd generation k-pop group', 'AI enthusiast']


In [4]:
class UserProfile(BaseModel):
    """Profile of a user.
    """
    user_name: str = Field(description="The user's preferred name")
    interests: list = Field(description="A list of the user's interests")

In [5]:
trustcall_extractor = create_extractor(
    llm,
    tools=[UserProfile],
    tool_choice="UserProfile",
)

In [6]:
MODEL_SYSTEM_MESSAGE = """
    You are a helpful assistant with memory that provides information about the user. 
    If you have memory for this user, use it to personalize your responses.
    Here is the memory (it may be empty): {memory}
"""

TRUSTCALL_INSTRUCTION = """
    Create or update the memory (JSON doc) to incorporate information from the following conversation:
"""

def call_model(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Load memory from the store and use it to personalize the chatbot's response.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")

    if existing_memory and existing_memory.value:
        memory_dict = existing_memory.value
        formatted_memory = (
            f"Name: {memory_dict.get('user_name', 'Unknown')}\n"
            f"Location: {memory_dict.get('user_location', 'Unknown')}\n"
            f"Interests: {', '.join(memory_dict.get('interests', []))}"      
        )
    else:
        formatted_memory = None

    system_msg = MODEL_SYSTEM_MESSAGE.format(memory=formatted_memory)

    response = llm.invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": response}

def write_memory(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Reflect on the chat history and save a memory to the store.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")
        
    existing_profile = {"UserProfile": existing_memory.value} if existing_memory else None
    
    result = trustcall_extractor.invoke({"messages": [SystemMessage(content=TRUSTCALL_INSTRUCTION)]+state["messages"], "existing": existing_profile})
    
    updated_profile = result["responses"][0].model_dump()

    key = "user_memory"
    store.put(namespace, key, updated_profile)

In [7]:
graph = StateGraph(MessagesState)
graph.add_node("call_model", call_model)
graph.add_node("write_memory", write_memory)
graph.add_edge(START, "call_model")
graph.add_edge("call_model", "write_memory")
graph.add_edge("write_memory", END)

across_thread_memory = InMemoryStore()
within_thread_memory = MemorySaver()
graph = graph.compile(checkpointer=within_thread_memory, store=across_thread_memory)


In [8]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

input_messages = [HumanMessage(content="Hi, my name is Yoona")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, my name is Yoona
================================== Ai Message ==================================

Hello Yoona! It's nice to meet you. How can I assist you today?


In [10]:
input_messages = [HumanMessage(content="I love 2nd generation k-pop group and AI enthusiast.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I love 2nd generation k-pop group and AI enthusiast.
================================== Ai Message ==================================

It seems like you mentioned your interests in 2nd generation K-pop groups and artificial intelligence. If you have any specific questions or if you'd like to discuss these topics further, feel free to let me know!


In [11]:
user_id = "1"
namespace = ("memory", user_id)
existing_memory = across_thread_memory.get(namespace, "user_memory")
existing_memory.dict()

{'value': {'user_name': 'Yoona',
  'interests': ['2nd generation K-pop groups', 'AI enthusiast']},
 'key': 'user_memory',
 'namespace': ['memory', '1'],
 'created_at': '2025-01-01T10:26:01.021240+00:00',
 'updated_at': '2025-01-01T10:26:01.021241+00:00'}

In [12]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}

input_messages = [HumanMessage(content="What book about AI do you recommend for me?")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What book about AI do you recommend for me?
================================== Ai Message ==================================

I recommend the book "Life 3.0: Being Human in the Age of Artificial Intelligence" by Max Tegmark. It explores the potential impact of artificial intelligence on humanity and society, covering various aspects of AI development and its implications. Given your interest in AI, I believe you will find it both informative and thought-provoking.
